# Build123d 基本動作確認ノートブック

このノートブックでは、Build123dのインストールと基本的な3Dモデル生成を確認します。

**流れ**
1. インストール
2. 基本形状の生成（Box, Cylinder, Sphere）
3. ブーリアン演算（足し算・引き算）
4. フィレット・面取り
5. 機械部品の例（フランジ付きシャフト）
6. ファイル出力（STEP / STL）
7. LLMプロンプトテンプレート

---
> **LLMを使って3Dモデルを生成したい場合は、セル7のプロンプトをコピーして使ってください。**

## 1. インストール

In [ ]:
# build123dのインストール（Colab初回実行時）
%pip install build123d
print("✅ build123d インストール完了")

In [ ]:
# インポートと動作確認
from build123d import *
import os

print("✅ build123d インポート成功")
print(f"   バージョン確認: Box(10,10,10) → {Box(10,10,10)}")

## 2. 基本形状の生成

In [ ]:
# ---- Box（直方体）----
with BuildPart() as box_part:
    Box(100, 50, 30)  # 幅100mm, 奥行50mm, 高さ30mm

print(f"✅ Box 生成完了")
print(f"   体積: {box_part.part.volume:.1f} mm³")
print(f"   バウンディングボックス: {box_part.part.bounding_box()}")

# ---- Cylinder（円柱）----
with BuildPart() as cyl_part:
    Cylinder(radius=25, height=80)  # 半径25mm, 高さ80mm

print(f"\n✅ Cylinder 生成完了")
print(f"   体積: {cyl_part.part.volume:.1f} mm³")

# ---- Sphere（球）----
with BuildPart() as sphere_part:
    Sphere(radius=20)  # 半径20mm

print(f"\n✅ Sphere 生成完了")
print(f"   体積: {sphere_part.part.volume:.1f} mm³")

## 3. ブーリアン演算（足し算・引き算）

In [ ]:
# ---- 足し算（Union）: BoxにCylinderを合体 ----
with BuildPart() as union_part:
    Box(60, 60, 20)
    Cylinder(radius=15, height=40)  # Boxの上にCylinderが生える

print(f"✅ Union（合体）生成完了")
print(f"   体積: {union_part.part.volume:.1f} mm³")

In [ ]:
# ---- 引き算（Subtract）: Boxに穴を開ける ----
with BuildPart() as hole_part:
    Box(80, 80, 30)
    with Locations((0, 0, 0)):
        Cylinder(radius=20, height=30, mode=Mode.SUBTRACT)  # 中心に穴

print(f"✅ Subtract（穴あき）生成完了")
print(f"   体積: {hole_part.part.volume:.1f} mm³")

# 参考：穴なしBoxの体積
vol_box = 80 * 80 * 30
vol_hole = 3.14159 * 20**2 * 30
print(f"   検算: Box体積 - 穴体積 ≈ {vol_box - vol_hole:.1f} mm³")

In [ ]:
# ---- 複数の穴：ボルト穴パターン ----
with BuildPart() as bolt_plate:
    Box(100, 100, 15)
    # 四隅にボルト穴（直径10mm = 半径5mm）
    with GridLocations(70, 70, 2, 2):  # 70mmピッチ、2×2グリッド
        Cylinder(radius=5, height=15, mode=Mode.SUBTRACT)

print(f"✅ ボルト穴プレート生成完了")
print(f"   体積: {bolt_plate.part.volume:.1f} mm³")

## 4. フィレット・面取り

In [ ]:
# ---- フィレット（角丸め）----
with BuildPart() as fillet_part:
    Box(80, 60, 25)
    fillet(fillet_part.edges(), radius=5)  # すべてのエッジをR5でフィレット

print(f"✅ フィレット生成完了")
print(f"   体積: {fillet_part.part.volume:.1f} mm³")

# ---- 面取り（Chamfer）----
with BuildPart() as chamfer_part:
    Box(80, 60, 25)
    chamfer(chamfer_part.edges(), length=3)  # すべてのエッジをC3で面取り

print(f"\n✅ 面取り生成完了")
print(f"   体積: {chamfer_part.part.volume:.1f} mm³")

## 5. 機械部品の例：フランジ付きシャフト

LLMが生成するコードの典型的な例です。

In [ ]:
# ---- フランジ付きシャフト ----
# パラメータ（LLMがここを変えて返す）
shaft_diameter   = 30    # シャフト直径 [mm]
shaft_length     = 120   # シャフト長さ [mm]
flange_diameter  = 80    # フランジ直径 [mm]
flange_thickness = 15    # フランジ厚さ [mm]
bolt_hole_dia    = 10    # ボルト穴直径 [mm]
bolt_pcd         = 60    # ボルト穴PCD [mm]
n_bolts          = 4     # ボルト穴数
fillet_r         = 3     # フィレット半径 [mm]

with BuildPart() as flange_shaft:
    # シャフト部
    Cylinder(radius=shaft_diameter/2, height=shaft_length)
    
    # フランジ部（シャフト底面に合体）
    with Locations((0, 0, -shaft_length/2 + flange_thickness/2)):
        Cylinder(radius=flange_diameter/2, height=flange_thickness)
    
    # ボルト穴（フランジに均等配置）
    with PolarLocations(bolt_pcd/2, n_bolts):
        Cylinder(radius=bolt_hole_dia/2, height=flange_thickness, mode=Mode.SUBTRACT)
    
    # シャフトとフランジの境界をフィレット
    fillet(
        flange_shaft.edges().filter_by_position(Axis.Z, -shaft_length/2 + flange_thickness, -shaft_length/2 + flange_thickness + 1),
        radius=fillet_r
    )

print(f"✅ フランジ付きシャフト生成完了")
print(f"   体積: {flange_shaft.part.volume:.1f} mm³")
print(f"   シャフト径: {shaft_diameter}mm, 長さ: {shaft_length}mm")
print(f"   フランジ径: {flange_diameter}mm, ボルト穴: {n_bolts}×φ{bolt_hole_dia}mm")

In [ ]:
# ---- もう一例：Lブラケット ----
width      = 60   # 幅 [mm]
height     = 80   # 高さ [mm]
depth      = 50   # 奥行き [mm]
thickness  = 8    # 板厚 [mm]

with BuildPart() as l_bracket:
    # 縦板
    Box(thickness, width, height)
    # 横板
    with Locations((depth/2, 0, -height/2 + thickness/2)):
        Box(depth, width, thickness)
    # エッジのフィレット
    fillet(l_bracket.edges().filter_by_position(Axis.X, 0, 1), radius=5)

print(f"✅ Lブラケット生成完了")
print(f"   体積: {l_bracket.part.volume:.1f} mm³")

## 6. ファイル出力（STEP / STL）

- **STEP**: FreeCADやSolidWorksで開ける汎用形式
- **STL**: 3Dプリントや後続のメッシャーへ渡す形式

In [ ]:
# 出力ディレクトリの作成
OUTPUT_DIR = "output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# フランジ付きシャフトを出力
step_path = f"{OUTPUT_DIR}/flange_shaft.step"
stl_path  = f"{OUTPUT_DIR}/flange_shaft.stl"

export_step(flange_shaft.part, step_path)
export_stl(flange_shaft.part, stl_path)

print(f"✅ STEP出力: {step_path}  ({os.path.getsize(step_path):,} bytes)")
print(f"✅ STL出力:  {stl_path}  ({os.path.getsize(stl_path):,} bytes)")

# Lブラケットも出力
export_step(l_bracket.part, f"{OUTPUT_DIR}/l_bracket.step")
export_stl(l_bracket.part,  f"{OUTPUT_DIR}/l_bracket.stl")
print(f"✅ Lブラケット STEP/STL出力完了")

print(f"\n📁 出力ファイル一覧:")
for f in os.listdir(OUTPUT_DIR):
    print(f"   {f}")

In [ ]:
# Colabからファイルをダウンロードする場合
try:
    from google.colab import files
    files.download(step_path)
    files.download(stl_path)
    print("✅ ダウンロード開始")
except ImportError:
    print("ℹ️ ローカル環境では上記のoutputフォルダを確認してください")

## 7. LLMプロンプトテンプレート

以下のプロンプトをChatGPT / Claude / Geminiなどにコピーして使ってください。  
`【作りたいもの】` の部分を書き換えるだけでBuild123dコードが生成されます。

In [ ]:
PROMPT_TEMPLATE = """
あなたはBuild123dのエキスパートです。以下のルールに従ってPythonコードを生成してください。

【ルール】
1. 必ず `from build123d import *` でインポートする
2. 形状は `with BuildPart() as part:` のコンテキストマネージャ内に書く
3. 寸法はすべてmm単位のfloatまたはintで書く
4. パラメータは変数として冒頭にまとめて定義する（変更しやすくするため）
5. ブーリアン演算は mode=Mode.SUBTRACT（穴・引き算）または mode=Mode.ADD（合体）を使う
6. スケッチで穴を作る場合は BuildSketch をネストせず、以下のように mode=Mode.SUBTRACT を使う：
   with BuildSketch() as sk:
       Circle(outer_r)
       Circle(inner_r, mode=Mode.SUBTRACT)
7. フィレットは `fillet(part.edges(), radius=R)` で指定する
8. 最後に以下の2行でファイル出力する：
   export_step(part.part, "output.step")
   export_stl(part.part, "output.stl")
9. コード以外の説明文は不要。コードブロックのみ返す。

【基本形状リファレンス】
- Box(length, width, height)
- Cylinder(radius=R, height=H)
- Sphere(radius=R)
- Cone(bottom_radius=R1, top_radius=R2, height=H)

【スケッチ押し出しリファレンス】
- 断面形状を作ってから押し出す方法（パイプ・複雑断面に使う）:
  with BuildSketch() as sk:
      Circle(outer_r)                        # 外形
      Circle(inner_r, mode=Mode.SUBTRACT)    # 内穴（BuildSketchのネストは使わない）
  extrude(amount=length)

【配置・パターン】
- Locations((x, y, z)): 指定座標に移動
- PolarLocations(radius, count): 円周上に均等配置
- GridLocations(x_spacing, y_spacing, x_count, y_count): グリッド配置

【よくある間違い・注意事項】
- BuildSketch を入れ子（ネスト）にしない
- 穴はすべて mode=Mode.SUBTRACT で指定する
- fillet はすべての形状が確定した後、最後に適用する

【作りたいもの】
"""

print("📋 以下をLLMにコピー＆ペーストして、最後に作りたいものを追記してください：")
print("="*60)
print(PROMPT_TEMPLATE)
print("="*60)
print("\n例：【作りたいもの】")
print("外径60mm、内径40mm、長さ100mmのパイプ。両端にR3のフィレットあり。")

## 8. LLMが生成したコードを実行するセル

LLMから返ってきたコードをここに貼り付けて実行してください。

In [ ]:
# ============================================================
# ↓ ここにLLMが生成したコードを貼り付けてください
# ============================================================

from build123d import *
import os
os.makedirs("output", exist_ok=True)

# --- LLM生成コードをここに貼る ---


# ============================================================
print("✅ 実行完了 / outputフォルダにSTEP・STLが保存されました")

---
## 補足メモ

**Colabでの3Dプレビュー**  
Colab上でのインタラクティブな3D表示は現時点では制限があります。  
STLファイルを出力して、別途Three.jsビューワーや外部ツール（MeshLab、FreeCAD等）で確認してください。

**ローカル環境でのプレビュー**  
VS Code + `ocp_vscode` 拡張機能を使うとリアルタイムで3D確認できます。
```
pip install ocp_vscode
```

**後続パーツとの連携**  
- STLファイル → pygmshメッシャーへ渡す  
- STEPファイル → FreeCADやCalculiXのCAEツールへ渡す

**参考リンク**  
- [Build123d 公式ドキュメント](https://build123d.readthedocs.io/)  
- [Build123d GitHub](https://github.com/gumyr/build123d)